# Load, cleanse and persist data using NumPy, Pandas and MySQL Connector/Python

In [1]:
#!pip install numpy pandas scikit-learn mysql-connector-python

import os
from pprint import pprint

import numpy as np
import pandas as pd
import mysql.connector as sq

from sklearn.preprocessing import LabelEncoder


## Load Data from CSV into Pandas DataFrame

In [2]:
path_to_raw_data_file = "./modified_kaggle_automobile_data.csv"

load_with_pandas = True  # Set to True to load data using Pandas, False to load using NumPy

if load_with_pandas:
    # Load the raw data from the CSV file into a Pandas DataFrame
    print("Loading raw data from CSV file into a Pandas DataFrame...")
    df = pd.read_csv(path_to_raw_data_file)
    df.info()
    df.head()
else:
    # Load the raw data from the CSV file into a NumPystructured array
    print("Loading raw data from CSV file into a NumPy structured array...")

    # genfromtxt() is more robust than loadtxt() as it can can handle CSV files with missing values, 
    # mixed data types (strings and numbers), and more complex structures.
    data_arr = np.genfromtxt(path_to_raw_data_file, delimiter=",", dtype=None, names=True, encoding="utf-8")
    print(type(data_arr))
    print()
    print(data_arr.shape)
    print()
    print(data_arr.dtype)
    print()
    print(data_arr[:5])
    print()

    # Convert the NumPy structured array to a Pandas DataFrame
    df = pd.DataFrame(data_arr)
    df.info()
    print()
    
    # Rename columns to match the original dataset's column names
    cols_to_rename = {
        "normalizedlosses": "normalized-losses",
        "fueltype": "fuel-type",
        "numofdoors": "num-of-doors",
        "bodystyle": "body-style",
        "drivewheels": "drive-wheels",
        "enginelocation": "engine-location",
        "wheelbase": "wheel-base",
        "curbweight": "curb-weight",
        "enginetype": "engine-type",
        "numofcylinders": "num-of-cylinders",
        "enginesize": "engine-size",
        "fuelsystem": "fuel-system",
        "compressionratio": "compression-ratio",
        "peakrpm": "peak-rpm",
        "citympg": "city-mpg",
        "highwaympg": "highway-mpg"
    }
    df.rename(columns=cols_to_rename, inplace=True)
    df.info()
    print()
    
    # Convert specified columns from int64 to float64
    cols_to_convert = ["symboling", "curb-weight", "city-mpg", "highway-mpg"]
    df[cols_to_convert] = df[cols_to_convert].astype("float64")

    # Replace 'NaN' string values in the 'price' column with a space character to avoid issues during conversion to numeric type
    df["price"] = pd.to_numeric(df["price"], errors='coerce')
    # df["price"] = df["price"].fillna(0)
    # df["price"] = df["price"].astype(int)

    df.info()
    df.head()


Loading raw data from CSV file into a Pandas DataFrame...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          200 non-null    float64
 1   normalized-losses  200 non-null    object 
 2   make               200 non-null    object 
 3   fuel-type          200 non-null    object 
 4   aspiration         200 non-null    object 
 5   num-of-doors       200 non-null    object 
 6   body-style         200 non-null    object 
 7   drive-wheels       200 non-null    object 
 8   engine-location    200 non-null    object 
 9   wheel-base         198 non-null    float64
 10  length             200 non-null    float64
 11  width              200 non-null    float64
 12  height             200 non-null    float64
 13  curb-weight        200 non-null    float64
 14  engine-type        200 non-null    object 
 15  num-of-cylinders

## Data Cleansing

### Col Headings

In [3]:
# Drop irrelevant cols from the DataFrame
print("\nCol names before dropping:\n", df.columns.tolist())
print("Number of cols before dropping:", len(df.columns))

cols_to_drop = ["symboling", "normalized-losses", "num-of-doors", "body-style", "drive-wheels", "engine-location", "wheel-base", "length", "width", "height", "city-mpg", "highway-mpg"]
df.drop(columns=cols_to_drop, inplace=True)

print("\nCol names after dropping:\n", df.columns.tolist())
print("Number of cols after dropping:", len(df.columns))

# Replace dashes with underscores in col headers
df.columns = df.columns.str.replace("-", "_")
print("\nUpdated Col names:\n", df.columns.tolist())

# Rename column(s)
df.rename(columns={"horsepower": "hp", "peak_rpm": "rpm"}, inplace=True)
col_list = df.columns.tolist()
print("\nUpdated Col List:\n", col_list)



Col names before dropping:
 ['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']
Number of cols before dropping: 26

Col names after dropping:
 ['make', 'fuel-type', 'aspiration', 'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'price']
Number of cols after dropping: 14

Updated Col names:
 ['make', 'fuel_type', 'aspiration', 'curb_weight', 'engine_type', 'num_of_cylinders', 'engine_size', 'fuel_system', 'bore', 'stroke', 'compression_ratio', 'horsepower', 'peak_rpm', 'price']

Updated Col List:
 ['make', 'fuel_type', 'aspiration', 'curb_weight', 'engine_type', 'num_of_cylinders', 'engine_

In [4]:
print()
df.info()
df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   make               200 non-null    object 
 1   fuel_type          200 non-null    object 
 2   aspiration         200 non-null    object 
 3   curb_weight        200 non-null    float64
 4   engine_type        200 non-null    object 
 5   num_of_cylinders   200 non-null    object 
 6   engine_size        197 non-null    float64
 7   fuel_system        200 non-null    object 
 8   bore               200 non-null    object 
 9   stroke             198 non-null    object 
 10  compression_ratio  200 non-null    float64
 11  hp                 199 non-null    object 
 12  rpm                200 non-null    object 
 13  price              198 non-null    object 
dtypes: float64(3), object(11)
memory usage: 22.6+ KB


,make,fuel_type,aspiration,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,hp,rpm,price
0,alfa-romero,gas,std,2548.0,dohc,four,130.0,mpfi,3.47,2.68,9.0,111,5000,13495
1,alfa-romero,gas,std,2548.0,dohc,four,130.0,mpfi,3.47,2.68,9.0,111,5000,16500
2,alfa-romero,gas,std,2823.0,ohcv,six,152.0,mpfi,2.68,3.47,9.0,154,5000,16500
3,audi,gas,std,2337.0,ohc,four,109.0,mpfi,3.19,3.4,10.0,102,5500,13950
4,audi,gas,std,2824.0,ohc,five,136.0,mpfi,3.19,3.4,8.0,115,5500,17450


### Data Cleansing - Data

In [5]:
# Drop completely empty rows from the DataFrame
print("Number of rows before dropping empty rows:", len(df))
df.dropna(how="all", inplace=True)
print("Number of rows after dropping empty rows:", len(df))

# Convert cells with "?" to NaN 
df.replace("?", np.nan, inplace=True)

# Convert cells with empty strings to NaN
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# Drop rows with NaN in any of the cols
print("\nNumber of rows before dropping rows with NaN in any col:", len(df))
df.dropna(how="any", inplace=True)
print("Number of rows after dropping rows with NaN in any col:", len(df))

# Drop rows that have "l" in the "engine_type" column
print("\nNumber of rows before dropping rows with 'l' in engine_type col:", len(df))
df.drop(df[df["engine_type"] == "l"].index, inplace=True)
print("Number of rows after dropping rows with 'l' in engine_type col:", len(df))

# Find and remove duplicate rows from the DataFrame
duplicates = df[df.duplicated()]
print("\nNumber of rows before removing duplicates:", len(df))
df.drop_duplicates(inplace=True)
print("Number of rows after removing duplicates:", len(df))


Number of rows before dropping empty rows: 205
Number of rows after dropping empty rows: 200

Number of rows before dropping rows with NaN in any col: 200
Number of rows after dropping rows with NaN in any col: 182

Number of rows before dropping rows with 'l' in engine_type col: 182
Number of rows after dropping rows with 'l' in engine_type col: 171

Number of rows before removing duplicates: 171
Number of rows after removing duplicates: 171


In [6]:
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 204
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   make               171 non-null    object 
 1   fuel_type          171 non-null    object 
 2   aspiration         171 non-null    object 
 3   curb_weight        171 non-null    float64
 4   engine_type        171 non-null    object 
 5   num_of_cylinders   171 non-null    object 
 6   engine_size        171 non-null    float64
 7   fuel_system        171 non-null    object 
 8   bore               171 non-null    object 
 9   stroke             171 non-null    object 
 10  compression_ratio  171 non-null    float64
 11  hp                 171 non-null    object 
 12  rpm                171 non-null    object 
 13  price              171 non-null    object 
dtypes: float64(3), object(11)
memory usage: 20.0+ KB


,curb_weight,engine_size,compression_ratio
count,171.000000,171.000000,171.000000
mean,2526.309942,127.555556,10.048655
std,498.440831,41.633995,3.881585
min,1713.000000,79.000000,7.000000
25%,2142.500000,98.000000,8.600000
50%,2410.000000,110.000000,9.000000
75%,2828.500000,141.000000,9.400000
max,4066.000000,326.000000,23.000000


In [7]:
# Change Data Types of Columns to Appropriate Types
df["curb_weight"] = df["curb_weight"].apply(np.floor).astype(int)

df["bore"] = df["bore"].astype(float)
df["stroke"] = df["stroke"].astype(float)

df["hp"] = df["hp"].astype(int)
df["rpm"] = df["rpm"].astype(int)
df["price"] = df["price"].astype(int)


In [8]:
df.info()
print()
print(df.head())
df.describe()


<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 204
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   make               171 non-null    object 
 1   fuel_type          171 non-null    object 
 2   aspiration         171 non-null    object 
 3   curb_weight        171 non-null    int64  
 4   engine_type        171 non-null    object 
 5   num_of_cylinders   171 non-null    object 
 6   engine_size        171 non-null    float64
 7   fuel_system        171 non-null    object 
 8   bore               171 non-null    float64
 9   stroke             171 non-null    float64
 10  compression_ratio  171 non-null    float64
 11  hp                 171 non-null    int64  
 12  rpm                171 non-null    int64  
 13  price              171 non-null    int64  
dtypes: float64(4), int64(4), object(6)
memory usage: 20.0+ KB

          make fuel_type aspiration  curb_weight engine_type num_of_

,curb_weight,engine_size,bore,stroke,compression_ratio,hp,rpm,price
count,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000
mean,2526.309942,127.555556,3.318772,3.254678,10.048655,103.122807,5110.526316,12997.233918
std,498.440831,41.633995,0.271216,0.298106,3.881585,37.613860,453.911449,8084.290103
min,1713.000000,79.000000,2.540000,2.070000,7.000000,52.000000,4200.000000,5118.000000
25%,2142.500000,98.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,7649.000000
50%,2410.000000,110.000000,3.270000,3.290000,9.000000,92.000000,5200.000000,9980.000000
75%,2828.500000,141.000000,3.580000,3.405000,9.400000,116.000000,5500.000000,15991.500000
max,4066.000000,326.000000,3.940000,4.170000,23.000000,262.000000,6600.000000,45400.000000


### Categorical Feature Cleansing

In [9]:
# Identify Categorical Columns in the DataFrame
categorical_cols = [col for col in df.columns if df[col].dtype == "object"]
print("\nCategorical Columns:\n", categorical_cols)

# Create a new DataFrame that contains only the categorical columns from the original DataFrame
categorical_df = df[categorical_cols]
categorical_df.head()

# Print the unique values for each categorical column in the new DataFrame
print()
for col in categorical_df.columns:
    print(f"{col}: \n{categorical_df[col].unique()}\n")



Categorical Columns:
 ['make', 'fuel_type', 'aspiration', 'engine_type', 'num_of_cylinders', 'fuel_system']

make: 
['alfa-romero' 'audi' 'bmw' 'chevrolet' 'dodge' 'honda' 'isuzu' 'jaguar'
 'mazda' 'mercedes-benz' 'mercury' 'mitsubishi' 'nissan' 'plymouth'
 'porsche' 'saab' 'subaru' 'toyota' 'volkswagen' 'volvo']

fuel_type: 
['gas' 'diesel']

aspiration: 
['std' 'turbo']

engine_type: 
['dohc' 'ohcv' 'ohc' 'ohcf']

num_of_cylinders: 
['four' 'six' 'five' 'twelve' 'eight']

fuel_system: 
['mpfi' '2bbl' 'mfi' '1bbl' 'spfi' 'idi' 'spdi']



In [10]:
# Map num_of_cylinders to numeric values in the categorical_df DataFrame
cylinder_mapping = {
    "four": 4,
    "five": 5,
    "six": 6,
    "eight": 8,
    "twelve": 12
}

# Replace the values in the num_of_cylinders column using the mapping
categorical_df.loc[:, "num_of_cylinders"] = categorical_df["num_of_cylinders"].map(cylinder_mapping)

# Change the data type of the num_of_cylinders column to int
categorical_df["num_of_cylinders"] = categorical_df["num_of_cylinders"].astype(int)

categorical_df.info()
categorical_df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 204
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   make              171 non-null    object
 1   fuel_type         171 non-null    object
 2   aspiration        171 non-null    object
 3   engine_type       171 non-null    object
 4   num_of_cylinders  171 non-null    int64 
 5   fuel_system       171 non-null    object
dtypes: int64(1), object(5)
memory usage: 9.4+ KB


/var/folders/xm/vwzlcq8n3n72g3j519crrygr0000gn/T/ipykernel_24073/579354713.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_df["num_of_cylinders"] = categorical_df["num_of_cylinders"].astype(int)


,make,fuel_type,aspiration,engine_type,num_of_cylinders,fuel_system
0,alfa-romero,gas,std,dohc,4,mpfi
1,alfa-romero,gas,std,dohc,4,mpfi
2,alfa-romero,gas,std,ohcv,6,mpfi
3,audi,gas,std,ohc,4,mpfi
4,audi,gas,std,ohc,5,mpfi


In [11]:
# Label encode make, fuel_type and aspiration cols
le = LabelEncoder()
categorical_df.loc[:, "make"] = le.fit_transform(categorical_df["make"])
categorical_df.loc[:, "fuel_type"] = le.fit_transform(categorical_df["fuel_type"])
categorical_df.loc[:, "aspiration"] = le.fit_transform(categorical_df["aspiration"]) 

# Change the data type of the make, fuel_type and aspiration columns to int
categorical_df["make"] = categorical_df["make"].astype(int)
categorical_df["fuel_type"] = categorical_df["fuel_type"].astype(int)
categorical_df["aspiration"] = categorical_df["aspiration"].astype(int)

categorical_df.info()
categorical_df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 204
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   make              171 non-null    int64 
 1   fuel_type         171 non-null    int64 
 2   aspiration        171 non-null    int64 
 3   engine_type       171 non-null    object
 4   num_of_cylinders  171 non-null    int64 
 5   fuel_system       171 non-null    object
dtypes: int64(4), object(2)
memory usage: 9.4+ KB


/var/folders/xm/vwzlcq8n3n72g3j519crrygr0000gn/T/ipykernel_24073/476753067.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_df["make"] = categorical_df["make"].astype(int)
/var/folders/xm/vwzlcq8n3n72g3j519crrygr0000gn/T/ipykernel_24073/476753067.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_df["fuel_type"] = categorical_df["fuel_type"].astype(int)
/var/folders/xm/vwzlcq8n3n72g3j519crrygr0000gn/T/ipykernel_24073/476753067.py:10: SettingWithCopyWarning: 
A value is trying to 

,make,fuel_type,aspiration,engine_type,num_of_cylinders,fuel_system
0,0,1,0,dohc,4,mpfi
1,0,1,0,dohc,4,mpfi
2,0,1,0,ohcv,6,mpfi
3,1,1,0,ohc,4,mpfi
4,1,1,0,ohc,5,mpfi


In [12]:
categorical_df.tail()


,make,fuel_type,aspiration,engine_type,num_of_cylinders,fuel_system
200,19,1,0,ohc,4,mpfi
201,19,1,1,ohc,4,mpfi
202,19,1,0,ohcv,6,mpfi
203,19,0,1,ohc,6,idi
204,19,1,1,ohc,4,mpfi


In [13]:
# One-hot encode the remaining categorical columns in the categorical_df DataFrame
one_hot_encode_cols = ["engine_type", "fuel_system"]
categorical_df = pd.get_dummies(categorical_df, columns=one_hot_encode_cols, dtype=int)

categorical_df.info()
categorical_df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 204
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   make              171 non-null    int64
 1   fuel_type         171 non-null    int64
 2   aspiration        171 non-null    int64
 3   num_of_cylinders  171 non-null    int64
 4   engine_type_dohc  171 non-null    int64
 5   engine_type_ohc   171 non-null    int64
 6   engine_type_ohcf  171 non-null    int64
 7   engine_type_ohcv  171 non-null    int64
 8   fuel_system_1bbl  171 non-null    int64
 9   fuel_system_2bbl  171 non-null    int64
 10  fuel_system_idi   171 non-null    int64
 11  fuel_system_mfi   171 non-null    int64
 12  fuel_system_mpfi  171 non-null    int64
 13  fuel_system_spdi  171 non-null    int64
 14  fuel_system_spfi  171 non-null    int64
dtypes: int64(15)
memory usage: 21.4 KB


,make,fuel_type,aspiration,num_of_cylinders,engine_type_dohc,engine_type_ohc,engine_type_ohcf,engine_type_ohcv,fuel_system_1bbl,fuel_system_2bbl,fuel_system_idi,fuel_system_mfi,fuel_system_mpfi,fuel_system_spdi,fuel_system_spfi
0,0,1,0,4,1,0,0,0,0,0,0,0,1,0,0
1,0,1,0,4,1,0,0,0,0,0,0,0,1,0,0
2,0,1,0,6,0,0,0,1,0,0,0,0,1,0,0
3,1,1,0,4,0,1,0,0,0,0,0,0,1,0,0
4,1,1,0,5,0,1,0,0,0,0,0,0,1,0,0


In [14]:
# Check for any remaining categorical columns in the categorical_df DataFrame
categorical_df_categorical_cols = [col for col in categorical_df.columns if categorical_df[col].dtype == "object"]
print("\nCategorical Columns:\n", categorical_df_categorical_cols)

# Check for any remaining categorical columns in the orignal DataFrame
df_categorical_cols = [col for col in df.columns if df[col].dtype == "object"]
print("\nCategorical Columns:\n", df_categorical_cols)



Categorical Columns:
 []

Categorical Columns:
 ['make', 'fuel_type', 'aspiration', 'engine_type', 'num_of_cylinders', 'fuel_system']


In [15]:
# Drop categorical columns from original DataFrame
numerical_df = df.drop(columns=df_categorical_cols)

# Check for any remaining categorical columns
numerical_df_categorical_cols = [col for col in numerical_df.columns if numerical_df[col].dtype == "object"]
print("\nCategorical Columns:\n", numerical_df_categorical_cols)

numerical_df.info()
numerical_df.head()



Categorical Columns:
 []
<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 204
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   curb_weight        171 non-null    int64  
 1   engine_size        171 non-null    float64
 2   bore               171 non-null    float64
 3   stroke             171 non-null    float64
 4   compression_ratio  171 non-null    float64
 5   hp                 171 non-null    int64  
 6   rpm                171 non-null    int64  
 7   price              171 non-null    int64  
dtypes: float64(4), int64(4)
memory usage: 12.0 KB


,curb_weight,engine_size,bore,stroke,compression_ratio,hp,rpm,price
0,2548,130.0,3.47,2.68,9.0,111,5000,13495
1,2548,130.0,3.47,2.68,9.0,111,5000,16500
2,2823,152.0,2.68,3.47,9.0,154,5000,16500
3,2337,109.0,3.19,3.40,10.0,102,5500,13950
4,2824,136.0,3.19,3.40,8.0,115,5500,17450


In [16]:
# Combine the numerical_df and categorical_df DataFrames into a single DataFrame
data_df = pd.concat([numerical_df, categorical_df], axis=1)
data_df.info()
data_df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 204
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   curb_weight        171 non-null    int64  
 1   engine_size        171 non-null    float64
 2   bore               171 non-null    float64
 3   stroke             171 non-null    float64
 4   compression_ratio  171 non-null    float64
 5   hp                 171 non-null    int64  
 6   rpm                171 non-null    int64  
 7   price              171 non-null    int64  
 8   make               171 non-null    int64  
 9   fuel_type          171 non-null    int64  
 10  aspiration         171 non-null    int64  
 11  num_of_cylinders   171 non-null    int64  
 12  engine_type_dohc   171 non-null    int64  
 13  engine_type_ohc    171 non-null    int64  
 14  engine_type_ohcf   171 non-null    int64  
 15  engine_type_ohcv   171 non-null    int64  
 16  fuel_system_1bbl   171 non-null

,curb_weight,engine_size,bore,stroke,compression_ratio,hp,rpm,price,make,fuel_type,...,engine_type_ohc,engine_type_ohcf,engine_type_ohcv,fuel_system_1bbl,fuel_system_2bbl,fuel_system_idi,fuel_system_mfi,fuel_system_mpfi,fuel_system_spdi,fuel_system_spfi
0,2548,130.0,3.47,2.68,9.0,111,5000,13495,0,1,...,0,0,0,0,0,0,0,1,0,0
1,2548,130.0,3.47,2.68,9.0,111,5000,16500,0,1,...,0,0,0,0,0,0,0,1,0,0
2,2823,152.0,2.68,3.47,9.0,154,5000,16500,0,1,...,0,0,1,0,0,0,0,1,0,0
3,2337,109.0,3.19,3.40,10.0,102,5500,13950,1,1,...,1,0,0,0,0,0,0,1,0,0
4,2824,136.0,3.19,3.40,8.0,115,5500,17450,1,1,...,1,0,0,0,0,0,0,1,0,0


In [17]:
# Save DataFrame to CSV
clean_data_file = "data.csv"
data_df.to_csv(clean_data_file, index=True, header=False)


## Load Cleansed data from CSV into MySQL (MySQL Connector/Python)

In [18]:
# Copy the cleansed data file to the MySQL data directory for loading into the database
# !rm -f /opt/homebrew/var/mysql/data.csv
!cp "./data.csv" /opt/homebrew/var/mysql/data.csv
!ls -al /opt/homebrew/var/mysql/data.csv


-rw-r--r--  1 rob  admin  12429 Sep 13 08:26 /opt/homebrew/var/mysql/data.csv


In [ ]:
"""
Set environment variables for MySQL connection (replace with your actual values)

These lines are for demonstration purposes only! 

In a real-world scenario, you would set these environment variables outside of your code, 
for example in your shell or CI/CD pipeline.
"""
# os.environ["MYSQL_HOST"] = "your_mysql_host"
# os.environ["MYSQL_USER"] = "your_mysql_user"
# os.environ["MYSQL_USER_PWD"] = "your_mysql_password"

In [20]:
#Get env vars
mysql_host = os.environ.get("MYSQL_HOST")
mysql_user = os.environ.get("MYSQL_USER")
mysql_user_password = os.environ.get("MYSQL_USER_PWD")

#Establish connection to DB
try:
    mydb=sq.connect(host=mysql_host,user=mysql_user,passwd=mysql_user_password, buffered=True)

    if mydb.is_connected():
        print("MySQL connection established successfully.")

        # Create Cursor - this will be used to execute SQL queries against the database
        mycursor = mydb.cursor()

        # Check if database exists
        mycursor.execute("SHOW DATABASES LIKE 'cleansed_automobobile_data'")
        db_exists = mycursor.fetchone()

        if not db_exists:
            print("Database 'cleansed_automobobile_data' does not exist.\nCreating database...")
            mycursor.execute("CREATE DATABASE cleansed_automobobile_data")
            print("Database 'cleansed_automobobile_data' created successfully.")

            # Switch to the 'cleansed_automobobile_data' database
            mycursor.execute("USE cleansed_automobobile_data")

            # Create the 'automobile_data' table in the 'cleansed_automobobile_data' database
            sql_str = """
            CREATE TABLE cleansed_automobobile_data.automobile_data (
                id INT NOT NULL,
                curb_weight INT NOT NULL,
                engine_size FLOAT NOT NULL,
                bore FLOAT NOT NULL,
                stroke FLOAT NOT NULL,
                compression_ratio FLOAT NOT NULL,
                hp INT NOT NULL,
                rpm INT NOT NULL,
                price INT NOT NULL,
                make INT NOT NULL,
                fuel_type INT NOT NULL,
                aspiration INT NOT NULL,
                num_of_cylinders INT NOT NULL,
                engine_type_dohc INT NOT NULL,
                engine_type_ohc INT NOT NULL,
                engine_type_ohcf INT NOT NULL,
                engine_type_ohcv INT NOT NULL,
                fuel_system_1bbl INT NOT NULL,
                fuel_system_2bbl INT NOT NULL,
                fuel_system_idi INT NOT NULL,
                fuel_system_mfi INT NOT NULL,
                fuel_system_mpfi INT NOT NULL,
                fuel_system_spdi INT NOT NULL,
                fuel_system_spfi INT NOT NULL,
                PRIMARY KEY(id)
            );
            """
            mycursor.execute(sql_str)

            # Describe the newly created table
            mycursor.execute("DESCRIBE automobile_data;")
            table_description = mycursor.fetchall()
            print("\nTable 'automobile_data' structure:")
            pprint(table_description)

            # Insert data from the CSV file into the newly created table using LOAD DATA INFILE
            sql_str = "LOAD DATA INFILE './data.csv' \
            INTO TABLE cleansed_automobobile_data.automobile_data \
            FIELDS TERMINATED BY ','  OPTIONALLY ENCLOSED BY '\"' LINES TERMINATED BY '\\n' IGNORE 1 ROWS;"
            mycursor.execute(sql_str)
            mydb.commit()

            sql_str = "select count(*) from cleansed_automobobile_data.automobile_data;"
            mycursor.execute(sql_str)
            count = mycursor.fetchone()[0]

            print(f"\nNumber of rows in cleansed_automobobile_data.automobile_data table: {count}")
        else:
            print("Database 'cleansed_automobobile_data' already exists.")

        mycursor.close()
        print("MySQL cursor closed successfully.")
    else:
        print("Failed to establish MySQL connection.")
except Exception as e:
    print(f"Error connecting to MySQL: {e}")


MySQL connection established successfully.
Database 'cleansed_automobobile_data' does not exist.
Creating database...
Database 'cleansed_automobobile_data' created successfully.

Table 'automobile_data' structure:
[('id', b'int', 'NO', bytearray(b'PRI'), None, ''),
 ('curb_weight', b'int', 'NO', bytearray(b''), None, ''),
 ('engine_size', b'float', 'NO', bytearray(b''), None, ''),
 ('bore', b'float', 'NO', bytearray(b''), None, ''),
 ('stroke', b'float', 'NO', bytearray(b''), None, ''),
 ('compression_ratio', b'float', 'NO', bytearray(b''), None, ''),
 ('hp', b'int', 'NO', bytearray(b''), None, ''),
 ('rpm', b'int', 'NO', bytearray(b''), None, ''),
 ('price', b'int', 'NO', bytearray(b''), None, ''),
 ('make', b'int', 'NO', bytearray(b''), None, ''),
 ('fuel_type', b'int', 'NO', bytearray(b''), None, ''),
 ('aspiration', b'int', 'NO', bytearray(b''), None, ''),
 ('num_of_cylinders', b'int', 'NO', bytearray(b''), None, ''),
 ('engine_type_dohc', b'int', 'NO', bytearray(b''), None, ''),
 

## Load data from DB into DataFrame

In [21]:
data_for_model_df = pd.read_sql("SELECT * FROM cleansed_automobobile_data.automobile_data", con=mydb)
data_for_model_df.drop(columns=["id"], inplace=True)

data_for_model_df.info()
print()
print(data_for_model_df.head())
data_for_model_df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   curb_weight        170 non-null    int64  
 1   engine_size        170 non-null    float64
 2   bore               170 non-null    float64
 3   stroke             170 non-null    float64
 4   compression_ratio  170 non-null    float64
 5   hp                 170 non-null    int64  
 6   rpm                170 non-null    int64  
 7   price              170 non-null    int64  
 8   make               170 non-null    int64  
 9   fuel_type          170 non-null    int64  
 10  aspiration         170 non-null    int64  
 11  num_of_cylinders   170 non-null    int64  
 12  engine_type_dohc   170 non-null    int64  
 13  engine_type_ohc    170 non-null    int64  
 14  engine_type_ohcf   170 non-null    int64  
 15  engine_type_ohcv   170 non-null    int64  
 16  fuel_system_1bbl   170 non

/var/folders/xm/vwzlcq8n3n72g3j519crrygr0000gn/T/ipykernel_24073/2145978793.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_for_model_df = pd.read_sql("SELECT * FROM cleansed_automobobile_data.automobile_data", con=mydb)


,curb_weight,engine_size,bore,stroke,compression_ratio,hp,rpm,price,make,fuel_type,...,engine_type_ohc,engine_type_ohcf,engine_type_ohcv,fuel_system_1bbl,fuel_system_2bbl,fuel_system_idi,fuel_system_mfi,fuel_system_mpfi,fuel_system_spdi,fuel_system_spfi
count,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,...,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000
mean,2526.182353,127.541176,3.317882,3.258059,10.054824,103.076471,5111.176471,12994.305882,11.811765,0.911765,...,0.788235,0.082353,0.070588,0.058824,0.335294,0.088235,0.005882,0.452941,0.052941,0.005882
std,499.910533,41.756565,0.271766,0.295682,3.892211,37.720084,455.172538,8108.081903,5.537380,0.284475,...,0.409766,0.275714,0.256892,0.235989,0.473488,0.284475,0.076696,0.499251,0.224578,0.076696
min,1713.000000,79.000000,2.540000,2.070000,7.000000,52.000000,4200.000000,5118.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2141.250000,98.000000,3.150000,3.112500,8.600000,70.000000,4800.000000,7629.000000,8.000000,1.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2407.500000,110.000000,3.270000,3.290000,9.000000,92.000000,5200.000000,9970.000000,12.000000,1.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2830.750000,141.000000,3.580000,3.407500,9.400000,116.000000,5500.000000,15994.750000,17.000000,1.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,4066.000000,326.000000,3.940000,4.170000,23.000000,262.000000,6600.000000,45400.000000,19.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
